# SVD-Grad-CAM


In [ ]:
import os
import numpy as np
import cv2
import random
import pandas as pd
from tqdm import tqdm
import keras.backend as K
import matplotlib.pyplot as plt
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from sklearn.decomposition import PCA
from keras import regularizers
from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot
from keras.applications.densenet import DenseNet169, DenseNet121, preprocess_input
from time import time

import datetime

In [ ]:
def listdir_nohidden(path):
    '''
    Utility function to find the list of files in a directory excluding the hidden files.
    Args:
        path: contains the path of the directory containing the images

    '''
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

In [ ]:
valid_image_df = pd.read_csv('/kaggle/input/testing/valid_image_data.csv', names=['Path','Count', 'Label'])
positives =0
i=0
for i, data in tqdm(valid_image_df.iterrows()):
    positives += data['Label']
    i+=1
    if i == 250:
        break
positives
dfs=valid_image_df['Path']

### DenseNet Model

In [ ]:
from tensorflow.keras.applications import DenseNet169,InceptionV3,ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def DenseNetModel(n_classes):
    base_model = DenseNet169(input_shape=(224, 224, 3),
                          weights='imagenet',
                          include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Global Average Pooling

    predictions = Dense(n_classes, activation='sigmoid')(x)  # Softmax activation for classification
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

n_classes = 1  # Number of classes for ImageNet dataset
model = DenseNetModel(1)
#model.summary()
layerName = 'relu'

In [ ]:
model.load_weights("/kaggle/input/my-model/MURA_MY_DenseNet-ELBOW.h5")

### VISUAL EXPLANATIONS
**Step 1: Image Preprocessing**

In [ ]:
def getImage2(path, size):
    '''
    Function to process the images
    Args:
        dataframe: contains the path to the images in the directory
        size: contains the value to which the shape of the image will resized
    '''
    Images = []
    try:
        image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
         # clipLimit -> Threshold for contrast limiting
        clahe = cv2.createCLAHE(clipLimit = 3,tileGridSize=(3,3))
        image = clahe.apply(image)
       # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        print(image.shape)
        # threshold
        thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
        hh, ww = thresh.shape
        print(thresh.shape)
        # make bottom 2 rows black where they are white the full width of the image
        thresh[hh-3:hh, 0:ww] = 0

        # get bounds of white pixels
        white = np.where(thresh==255)
        xmin, ymin, xmax, ymax = np.min(white[1]), np.min(white[0]), np.max(white[1]), np.max(white[0])
        #print(xmin,xmax,ymin,ymax)

        # crop the image at the bounds adding back the two blackened rows at the bottom
        crop = image[ymin:ymax+3, xmin:xmax]
        crop = cv2.resize(crop,(size,size))
        # Normalixation

        #crop=crop/255

        #image = randome_rotation_flip(image,size)
        print(crop.shape)
        rgb_array = np.stack((crop,) * 3, axis=-1)
        Images.append(rgb_array)


    except Exception as e:
        print(str(e))

    Images = np.asarray(Images).astype('float32')

   # mean = np.mean(Images)
   # std = np.std(Images)
    Images = Images/255

    #if K.image_data_format() == "channels_first":
    #plt.imshow(Images[0]) #Extended dimension 3
    #if K.image_data_format() == "channels_last":
    #  Images = np.expand_dims(Images,axis=3)             #Extended dimension 3(usebackend tensorflow:aixs=3; theano:axixs=1)
    return Images


Choose an Image


In [ ]:
import pandas as pd
helps= {'Path': ['/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11815/study1_positive/image2.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11756/study1_positive/image3.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11389/study1_positive/image4.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11823/study1_positive/image2.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11832/study1_positive/image1.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11839/study1_positive/image3.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11842/study1_positive/image5.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11823/study1_positive/image2.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11832/study1_positive/image1.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11472/study1_positive/image1.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11603/study1_positive/image1.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11780/study1_positive/image1.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11326/study1_positive/image1.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11326/study1_positive/image3.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11815/study1_positive/image2.png','/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11186/study1_positive/image2.png']}
dfs=pd.DataFrame(helps)

In [ ]:
img_path='/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11372/study1_negative/image1.png'
image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
clahe = cv2.createCLAHE(clipLimit = 3,tileGridSize=(3,3))
image = clahe.apply(image)

final_img = getImage2(img_path,224)
#x=final_img[0]
plt.imshow(final_img[0])
print(final_img.shape)

**Step 3 : to get SuperImposition of Class Activation Mappings on Input Image**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Model
import cv2
from tensorflow.keras.preprocessing import image
from PIL import Image

def show_imgwithheat(img_path, heatmap, alpha=0.4, return_array=False):
    """Show the image with heatmap.

    Args:
        img_path: string.
        heatmap: image array, get it by calling grad_cam().
        alpha: float, transparency of heatmap.
        return_array: bool, return a superimposed image array or not.
    Return:
        None or image array.
    """
    image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    # clipLimit -> Threshold for contrast limiting
    clahe = cv2.createCLAHE(clipLimit = 3,tileGridSize=(3,3))
    image = clahe.apply(image)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #print(heatmap.shape)
    #plt.imshow(heatmap)
    # threshold
    thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    hh, ww = thresh.shape
    # make bottom 2 rows black where they are white the full width of the image
    thresh[hh-3:hh, 0:ww] = 0

    # get bounds of white pixels
    white = np.where(thresh==255)
    xmin, ymin, xmax, ymax = np.min(white[1]), np.min(white[0]), np.max(white[1]), np.max(white[0])
        #print(xmin,xmax,ymin,ymax)

        # crop the image at the bounds adding back the two blackened rows at the bottom
    crop = image[ymin:ymax+3, xmin:xmax]
    img = cv2.resize(crop,(224,224))

 #Images = np.expand_dims(Images,axis=3) #Extended dimension 3
   # display(np.expand_dims(img,axis=2))
  #  img = cv2.imread(img_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = (heatmap*255).astype("uint8")
   # cv2.apply
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    superimposed_img = np.clip(superimposed_img, 0, 255).astype("uint8")
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)

    imgwithheat = Image.fromarray(superimposed_img)
    try:
        display(imgwithheat)
    except NameError:
        imgwithheat.show()

    if return_array:
        return superimposed_img

def prediction_label(p):
    if(p > 0.5):
        return 1
    else:
        return 0


# Last Conv Layer

Resnet50 'conv5_block3_out'

InceptionV3 'mixed10'

Densenet169 'relu'

In [ ]:
layerName='relu'

**GRAD-CAM**

In [ ]:
 import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.applications.imagenet_utils import decode_predictions

def svd_grad_cam(model, img, layer_name="relu", top_k=5, n_components=1):
    """
    Get heatmaps by Grad-CAM with SVD for the top K predicted classes.

    Args:
        model: A model object, built from tf.keras 2.X.
        img: An image ndarray.
        layer_name: A string, layer name in the model.
        top_k: An integer, number of top classes to generate heatmaps for.
        n_components: Number of components to keep in the SVD.

    Returns:
        A tuple of heatmaps and decoded predictions.
    """
    img_tensor = np.expand_dims(img, axis=0)

    conv_layer = model.get_layer(layer_name)
    heatmap_model = Model([model.inputs], [conv_layer.output, model.output])

    # Making the GradientTape persistent
    with tf.GradientTape(persistent=True) as gtape:
        conv_output, predictions = heatmap_model(img_tensor[0])
        sorted_indices = np.argsort(np.squeeze(predictions))[::-1]
        top_k_indices = sorted_indices[:top_k]

        heatmaps = np.zeros((top_k, conv_output.shape[1], conv_output.shape[2]))

        for i, category_id in enumerate(top_k_indices):
            output = predictions[:, category_id]
            grads = gtape.gradient(output, conv_output)
            grads_flat = np.array(grads).flatten().reshape(1664, 7, 7)
            grads_flat[np.isnan(grads_flat)] = 0

            X_svd = []
            

            for j in range(1664):
                s, u, vt = tf.linalg.svd(grads_flat[j], full_matrices=True)
                u = u[:, :2]
                s_top1 = s[:2]
                vt = vt[:2, :]
                s_top1_diag = tf.linalg.diag(s_top1)
                reconstructed_matrix = tf.matmul(u, tf.matmul(s_top1_diag, vt))
                X_svd.append(tf.reduce_mean(reconstructed_matrix))

            heatmap = tf.reduce_mean(tf.multiply(np.array(X_svd), conv_output), axis=-1)
            heatmap = np.maximum(heatmap, 0)
            max_heat = np.max(heatmap)
            if max_heat == 0:
                max_heat = 1e-10
            heatmap /= max_heat
            heatmaps[i, :, :] = np.squeeze(heatmap)

    # Deleting the tape after use since it's persistent
    del gtape

    decoded_predictions = decode_predictions(np.array(predictions), top=top_k)[0]
    return heatmaps, decoded_predictions


# Example usage:
heatmaps, decoded_predictions = svd_grad_cam(model, img_array, layer_name='relu', top_k=5)


In [ ]:
img_path='/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11375/study1_negative/image2.png'
imge = final_img = getImage3(img_path,224)
#x=final_img[0]
print(final_img.shape)

In [ ]:
grad_cams=grad_cam(model,final_img,label_name = ['negative','positive'],layer_name=layerName)
grad_cam_superimposed = imgwithheat(img_path, grad_cams[0])
gcc =coherency(grad_cams[0],getImage3(imge,224,1,grad_cam_superimposed),model,grad_cam,grad_cams[1])

guass_cam = svd__cam(model,final_img,label_name=['negative','positive'],layer_name=layerName)
guass_cam_superimposed = imgwithheat(img_path,guass_cam[0])
ggc=coherency(guass_cam[0],getImage3(imge,224,1,guass_cam_superimposed),model,svd__cam,guass_cam[1])

smooth_cam=eigen_cam(model,final_img,label_name = ['negative','positive'],layer_name=layerName)
smooth_superimposed = imgwithheat(img_path, smooth_cam[0])
sgc=coherency(smooth_cam[0],getImage3(imge,224,1,smooth_superimposed),model,eigen_cam,smooth_cam[1])

score_cams=score_camp(model,final_img,label_name = ['negative','positive'],layer_name=layerName)
print("00",score_cams[0].shape)
score_cam_superimposed = imgwithheat(img_path,score_cams[0])
sc=coherency(score_cams[0],getImage3(imge,224,1,score_cam_superimposed),model,score_camp,score_cams[1])

fig, ax = plt.subplots(nrows=1,ncols=5, figsize=(18, 11))
ax[0].imshow(final_img[0])
ax[0].set_title("input image")
ax[0].annotate("Coherency", xy=(0.5, -0.1), xycoords="axes fraction", ha="center", va="center", fontsize=10)
ax[1].imshow(grad_cam_superimposed)
ax[1].annotate("Coherency: "+tf.strings.as_string(gcc[2]).numpy().decode('utf-8'), xy=(0.5, -0.1), xycoords="axes fraction", ha="center", va="center", fontsize=12)
ax[1].set_title("Grad-CAM")
ax[2].imshow(smooth_superimposed)
ax[2].annotate("Coherency: "+tf.strings.as_string(sgc[2]).numpy().decode('utf-8'), xy=(0.5, -0.1), xycoords="axes fraction", ha="center", va="center", fontsize=12)
ax[2].set_title("Eigen_cam")
ax[3].imshow(guass_cam_superimposed)
ax[3].annotate("Coherency: "+tf.strings.as_string(ggc[2]).numpy().decode('utf-8'), xy=(0.5, -0.1), xycoords="axes fraction", ha="center", va="center", fontsize=12)
ax[3].set_title("Grad_new-CAM")
ax[4].imshow(score_cam_superimposed)
ax[4].annotate("Coherency: "+tf.strings.as_string(sc[2]).numpy().decode('utf-8'), xy=(0.5, -0.1), xycoords="axes fraction", ha="center", va="center", fontsize=12)
ax[4].set_title("Score-Cam")
for j in range(5):
    ax[j].axis('off')
plt.show()
fig.savefig('Coherency_DenseNet169')

# QUALITATIVE EVALUATION OF CAM TECHNIQUES

# **Average Drop.**
It measures the average percentage drop in
confidence for the target class c when the model sees only
the explanation map, instead of the full image. For one im-
age, the metric is defined as 
> ***(max(0, yc − oc)/yc) · 100***

where yc is the output score for class c when using the full
image, and oc the output score when using the explanation
map. The value is then averaged over a set of images.
# **Average Increase.** 
It computes, instead, the number of
times the confidence of the model is higher when using the
explanation map compared to when using the entire image.
Formally, for a single image it is defined as 
> ***1✶yc<oc · 100***

where ✶ is the indicator function. The value is then again
averaged over different images.
Insertion and Deletion. Deletion 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def average_drop(final_img, explanation_map, model, out, class_idx=None):
    # print(explanation_map.shape)
    # Get the output of the neural network on the explanation map
    out_on_exp = model.predict(explanation_map)[0]
    
    # Get the confidence of the correct class in the original input image
    confidence_on_inp = out[0]

    # Get the confidence of the correct class in the image after applying the explanation map
    confidence_on_exp = out_on_exp[0]

    # Compute the average drop in confidence
    average_drop = tf.math.maximum(0., confidence_on_inp - confidence_on_exp) / confidence_on_inp
    average_inc = 1 if ((confidence_on_inp - confidence_on_exp) < 0) else 0
    #print(average_drop,average_inc)
    return [average_drop, average_inc]

In [ ]:
def pearson_corr(saliency_map_A, saliency_map_B):
    # Compute the mean of each saliency map
    mean_A = tf.math.reduce_mean(saliency_map_A)
    mean_B = tf.math.reduce_mean(saliency_map_B)

    # Center the saliency maps
    saliency_map_A_centered = saliency_map_A - mean_A
    saliency_map_B_centered = saliency_map_B - mean_B
    
    # Compute the standard deviation of each saliency map
    std_A = tf.math.sqrt(tf.math.reduce_mean(tf.math.square(saliency_map_A_centered)))
    std_B = tf.math.sqrt(tf.math.reduce_mean(tf.math.square(saliency_map_B_centered)))

    # Compute the numerator of the Pearson correlation coefficient
    numerator = tf.reduce_mean(saliency_map_A_centered * saliency_map_B_centered - std_A*std_A*std_B*std_B)

    # Compute the denominator of the Pearson correlation coefficient
    denominator = std_A * std_B

    # Compute the Pearson correlation coefficient
    if std_A ==0 or std_B ==0:
        pearson_corr=numerator
    else:
        pearson_corr = numerator / denominator

    return pearson_corr

# **Maximum Coherency**
The CAM should contain all the
relevant features that explain a prediction and should re-
move useless features in a coherent way. As a consequence,
given an input image x and a class of interest c, the CAM
of x should not change when conditioning x on the CAM
itself. Formally,
> ***CAMc(x ⊙ CAMc(x)) = CAMc(x)***

Notice that this is equivalent to requiring that the CAM of
one image should be equal to that of the explanation map
obtained with the same CAM approach. To measure the
extent to which an approach satisfies the coherency prop-
erty, we define a metric that measures how much the CAM
changes when smoothing pixels with a low attribution score.
Following previous works in the comparison of saliency
maps [24, 5, 6, 2, 7, 8], we use the Pearson Correlation Co-
efficient between the two CAMs considered in Eq. 2:
> ***Coherency(x) = Cov(CAMc(x ⊙ CAMc(x)), CAMc(x)) / σCAMc(x⊙CAMc(x))σCAMc(x)***

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras
from tensorflow.keras import layers,backend
def coherency(saliency_map, explanation_map, model, attr_method,inp_score=[1]):
    # Convert explanation map to a tensor suitable for Keras
    # Compute the saliency map for the explanation map
    saliency_map_B = attr_method(model,explanation_map,label_name = ['negative','positive'],layer_name=layerName)
    # Flatten the saliency maps
    drop = max(0,inp_score[0]-saliency_map_B[1][0])
    inc=1
    if drop > 0 :
        inc=0
    #print(saliency_map.shape,saliency_map_B[0].shape)
    saliency_map_A2 = tf.reshape(saliency_map, [-1])
    saliency_map_B2 = tf.reshape(saliency_map_B[0], [-1])
    #print(saliency_map_B[0].shape,saliency_map_B2.shape)
    # Compute the Pearson correlation coefficient
    pearso_corr = pearson_corr(saliency_map_A2, saliency_map_B2)
    #print(pearso_corr)
    # Normalize the correlation coefficient to be between 0 and 1
    normalized_corr = (pearso_corr + 1) / 2

    return [drop,inc,normalized_corr]

# **Minimum Complexity**
 Beyond requiring that the CAM
should be coherent in removing features from the input im-
age, we must also require it to be as less complex as possi-
ble, i.e., it must contain the minimum set of pixels that ex-
plains the prediction. Employing the L1 norm as a proxy of
the complexity of a CAM, we define the Complexity mea-
sure as:
> ***Complexity(x) = ‖CAMc(x)‖1***

Complexity is minimized when the number of pixels high-
lighted by the attribution method is low.

In [ ]:
def complexity(saliency_map):
    return abs(saliency_map).sum()/(saliency_map.shape[-1]*saliency_map.shape[-2])

In [ ]:
results=outs

In [ ]:
results={'cam': {"Avg Drop": 0,"Avg Increase":0,"Coherency":0,"Complexicity" : 0},
             'grad-cam': {"Avg Drop": 0,"Avg Increase":0,"Coherency":0,"Complexicity" : 0},
             'scorecam': {"Avg Drop": 0,"Avg Increase":0,"Coherency":0,"Complexicity" : 0},
             'svd_grad_cam':{"Avg Drop": 0,"Avg Increase":0,"Coherency":0,"Complexicity" : 0},
             'eigen_cam':{"Avg Drop": 0,"Avg Increase":0,"Coherency":0,"Complexicity" : 0}
            }
def Meterics(dataframe, size):
    '''
    Function to process the images
    Args:
        dataframe: contains the path to the images in the directory
        size: contains the value to which the shape of the image will resized
    '''
    ko=0
    for i, data in tqdm(dataframe.iterrows()):
        imge=data['Path']
        print(imge)
        final_image = getImage3(imge,224)
        #heatmaps
        heatmap1=cam(model, final_image,
                   label_name = ['negative','positive'],layer_name=layerName)
        heatmap2=grad_cam(model, final_image,
                   label_name = ['negative','positive'],layer_name=layerName)
        heatmap3=score_camp(model, final_image,
                   label_name = ['negative','positive'],layer_name=layerName)
        heatmap4 = svd_grad_cam(model,final_image,
                           label_name=['negative','positive'],layer_name=layerName)
        heatmap5=eigen_cam(model,final_image,
                          label_name=['negative','positive'],layer_name=layerName)
        #Explnation_maps
        explnationmap1=imgwithheat(imge, heatmap1[0])
        explnationmap2=imgwithheat(imge, heatmap2[0])
        explnationmap3=imgwithheat(imge, heatmap3[0])
        explnationmap4=imgwithheat(imge, heatmap4[0])
        explnationmap5=imgwithheat(imge, heatmap5[0])

        preproccessed1=getImage3(imge,224,1,explnationmap1)
        preproccessed2=getImage3(imge,224,1,explnationmap2)
        preproccessed3=getImage3(imge,224,1,explnationmap3)
        preproccessed4=getImage3(imge,224,1,explnationmap4)
        preproccessed5=getImage3(imge,224,1,explnationmap5)
        #show_imgwithheat(imge,heatmap[0])

        avg=coherency(heatmap1[0],preproccessed1,model,cam,heatmap1[1])
        results['cam']["Avg Drop"]+=np.array(avg[0])
        results['cam']["Avg Increase"]+=avg[1]*100/size
        results['cam']["Coherency"]+=np.array(avg[2])*100/size
        results['cam']["Complexicity"]+=np.array(complexity(heatmap1[0]))*100/size

        avg=coherency(heatmap2[0],preproccessed2,model,grad_cam,heatmap2[1])
        results['grad-cam']["Avg Drop"]+=np.array(avg[0])
        results['grad-cam']["Avg Increase"]+=avg[1]*100/size
        results['grad-cam']["Coherency"]+=np.array(avg[2])*100/size
        results['grad-cam']["Complexicity"]+=np.array(complexity(heatmap2[0]))*100/size

        avg=coherency(heatmap3[0],preproccessed3,model,score_camp,heatmap3[1])
        results['scorecam']["Avg Drop"]+=np.array(avg[0])
        results['scorecam']["Avg Increase"]+=avg[1]*100/size
        results['scorecam']["Coherency"]+=np.array(avg[2])*100/size
        results['scorecam']["Complexicity"]+=np.array(complexity(heatmap3[0]))*100/size

        avg=coherency(heatmap4[0],preproccessed4,model,svd_grad_cam,heatmap4[1])
        results['svd_grad_cam']["Avg Drop"]+=np.array(avg[0])
        results['svd_grad_cam']["Avg Increase"]+=avg[1]*100/size
        results['svd_grad_cam']["Coherency"]+=np.array(avg[2])*100/size
        results['svd_grad_cam']["Complexicity"]+=np.array(complexity(heatmap4[0]))*100/size

        avg=coherency(heatmap5[0],preproccessed5,model,eigen_cam,heatmap5[1])
        results['eigen_cam']["Avg Drop"]+=np.array(avg[0])
        results['eigen_cam']["Avg Increase"]+=avg[1]*100/size
        results['eigen_cam']["Coherency"]+=np.array(avg[2])*100/size
        results['eigen_cam']["Complexicity"]+=np.array(complexity(heatmap5[0]))*10/size

        ko+=1
        if ko==size:
            break
    results['cam']["Avg Drop"]=results['cam']["Avg Drop"]*100/(size-results['cam']["Avg Increase"]*size/100)
    results['grad-cam']["Avg Drop"]/=(size-results['grad-cam']["Avg Increase"]*size/100)/100
    results['scorecam']["Avg Drop"]/=(size-results['scorecam']["Avg Increase"]*size/100)/100
    results['svd_grad_cam']["Avg Drop"]/=(size-results['svd_grad_cam']["Avg Increase"]*size/100)/100
    results['eigen_cam']["Avg Drop"]/=(size-results['eigen_cam']["Avg Increase"]*size/100)/100
    return results
outs=Meterics(dfs,15)

In [ ]:
outs

In [ ]:
df = pd.DataFrame(outs)

# Save the DataFrame to a CSV file
df.to_csv('output.csv', index=False)

In [ ]:
import pandas as pd
helps= {'Path': ['/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11815/study1_positive/image2.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11756/study1_positive/image3.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11389/study1_positive/image4.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11823/study1_positive/image2.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11832/study1_positive/image1.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11839/study1_positive/image3.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11842/study1_positive/image5.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11823/study1_positive/image2.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11832/study1_positive/image1.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11472/study1_positive/image1.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11603/study1_positive/image1.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11780/study1_positive/image1.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11326/study1_positive/image1.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11326/study1_positive/image3.png',
 '/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11815/study1_positive/image2.png','/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11603/study1_positive/image1.png']}
dfs=pd.DataFrame(helps)

In [ ]:
import numpy as np

def insertion(y_true, y_pred, heatmap):
    """
    Calculates the rise in the target class probability as pixels are added
    according to the CAM.

    Args:
        y_true (np.ndarray): The ground truth class labels.
        y_pred (np.ndarray): The predicted class labels.
        heatmap (np.ndarray): The CAM heatmap.

    Returns:
        float: The rise in the target class probability.
    """
    # Sort the heatmap in descending order
    heatmap_sorted = np.sort(heatmap)

    # Calculate the AUC for the full image
    auc_full = roc_auc_score(y_true, y_pred)

    # Calculate the AUC for each level of the heatmap
    aucs = []
    for threshold in heatmap_sorted:
        y_pred_threshold = y_pred[heatmap >= threshold]
        auc = roc_auc_score(y_true, y_pred_threshold)
        aucs.append(auc)

    # Calculate the insertion metric
    insertion = auc_full - np.mean(aucs)
    return insertion

def deletion(y_true, y_pred, heatmap):
    """
    Calculates the drop in the probability of the target class as
    important pixels (given by the CAM) are gradually removed from the image.

    Args:
        y_true (np.ndarray): The ground truth class labels.
        y_pred (np.ndarray): The predicted class labels.
        heatmap (np.ndarray): The CAM heatmap.

    Returns:
        float: The drop in the probability of the target class.
    """
    # Sort the heatmap in ascending order
    heatmap_sorted = np.sort(heatmap)[::-1]

    # Calculate the AUC for the full image
    auc_full = roc_auc_score(y_true, y_pred)

    # Calculate the AUC for each level of the heatmap
    aucs = []
    for threshold in heatmap_sorted:
        y_pred_threshold = y_pred[heatmap <= threshold]
        auc = roc_auc_score(y_true, y_pred_threshold)
        aucs.append(auc)

    # Calculate the deletion metric
    deletion = auc_full - np.mean(aucs)
    return deletion


In [ ]:
insertion(1,smooth_cam[1],smooth_cam[0])

In [ ]:
coh1 = [0,0,0,0]
coh2=[0,0,0,0]
def helper(final_img,img_path,ko):
    
    cams=cam(model,final_img,label_name = ['negative','positive'],layer_name=layerName)
    cam_superimposed = imgwithheat(img_path, cams[0])
    
    grad_cams=grad_cam(model,final_img,label_name = ['negative','positive'],layer_name=layerName)
    grad_cam_superimposed = imgwithheat(img_path, grad_cams[0])
    
    smooth_grad_cams=smoothgradNow(model,final_img,label_name = ['negative','positive'],layer_name=layerName)
    smooth_grad_cam_superimposed = imgwithheat(img_path, smooth_grad_cams[0])
    
    score_cams=score_camp(model,final_img,label_name = ['negative','positive'],layer_name=layerName)
    score_cam_superimposed = imgwithheat(img_path, score_cams[0])
    
    eigen_cams=eigen_cam(model,final_img,label_name = ['negative','positive'],layer_name=layerName)
    eigen_cam_superimposed = imgwithheat(img_path,eigen_cams[0])

    svd_grad_cams=svd_grad_cam(model,final_img,label_name = ['negative','positive'],layer_name=layerName)
    svd_grad_superimposed = imgwithheat(img_path, svd_grad_cams[0])

    fig, ax = plt.subplots(nrows=1,ncols=7, figsize=(20, 6))
    ax[0].imshow(final_img[0])
    ax[0].set_title("input image")
    ax[1].imshow(cam_superimposed)
    ax[1].set_title("CAM")
    ax[2].imshow(grad_cam_superimposed)
    ax[2].set_title("Grad-CAM")
    ax[3].imshow(smooth_grad_cam_superimposed)
    ax[3].set_title("Smooth-Grad-CAM")
    ax[4].imshow(score_cam_superimposed)
    ax[4].set_title("Score-CAM")
    ax[5].imshow(eigen_cam_superimposed)
    ax[5].set_title("Eigen-CAM")
    ax[6].imshow(svd_grad_superimposed)
    ax[6].set_title("SVD_Grad_CAM")
    for j in range(7):
        ax[j].axis('off')
    plt.show()
    rc='SVD Evalation'+str(ko)+'.png'
    fig.savefig(rc, bbox_inches='tight')

In [ ]:
helper(final_img,img_path,0)

In [ ]:
def getImagerp(dataframe, size):
    '''
    Function to process the images
    Args:
        dataframe: contains the path to the images in the directory
        size: contains the value to which the shape of the image will resized
    '''
    ko=1
    for i, data in tqdm(dataframe.iterrows()):
        img=data['Path']
        print(img)
        final_ = getImage3(img,224)
        helper(final_,img,ko)
        ko+=1
getImagerp(dfs,15)

In [ ]:
coh1

In [ ]:
coh2

# Selected Examples

/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11815/study1_positive/image2.png


/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11756/study1_positive/image3.png

/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11389/study1_positive/image4.png

/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11823/study1_positive/image2.png

/kaggle/input/my-mura/MURA-v1.1/valid/XR_ELBOW/patient11832/study1_positive/image1.png

In [ ]:
cams = cam(model,final_img,label_name=['negative','positive'],layer_name=layerName)
cam_superimposed = imgwithheat(img_path,cams[0])
c=coherency(cams[0],getImage3(imge,224,1,cam_superimposed),model,cam,cams[1])

grad_cams = grad_cam(model,final_img,label_name=['negative','positive'],layer_name=layerName)
grad_cam_superimposed = imgwithheat(img_path,grad_cams[0])
gc=coherency(grad_cams[0],getImage3(imge,224,1,grad_cam_superimposed),model,grad_cam,grad_cams[1])

smooth_cam=smoothgradNow(model,final_img,label_name = ['negative','positive'],layer_name=layerName)
smooth_superimposed = imgwithheat(img_path, smooth_cam[0])
sgc=coherency(smooth_cam[0],getImage3(imge,224,1,smooth_superimposed),model,smoothgradNow,smooth_cam[1])

score_cams=score_camp(model,final_img,label_name = ['negative','positive'],layer_name=layerName)
#print("00",score_cams[0].shape)
score_cam_superimposed = imgwithheat(img_path,score_cams[0])
sc=coherency(score_cams[0],getImage3(imge,224,1,score_cam_superimposed),model,score_camp,score_cams[1])

eigen_cams=eigen_cam(model,final_img,label_name = ['negative','positive'],layer_name=layerName)
eigen_cam_superimposed = imgwithheat(img_path,eigen_cams[0])
ec=coherency(eigen_cams[0],getImage3(imge,224,1,eigen_cam_superimposed),model,eigen_cam,score_cams[1])

svd_grad_cams = svd_grad_cam(model,final_img,label_name=['negative','positive'],layer_name=layerName)
svd_grad_cam_superimposed = imgwithheat(img_path,svd_grad_cams[0])
svgc=coherency(svd_grad_cams[0],getImage3(imge,224,1,svd_grad_cam_superimposed),model,svd_grad_cam,svd_grad_cams[1])

fig, ax = plt.subplots(nrows=1,ncols=7, figsize=(20, 13))
ax[0].imshow(final_img[0])
ax[0].annotate("Input Image", xy=(0.5, -0.2), xycoords="axes fraction", ha="center", va="center", fontsize=12)
ax[1].imshow(cam_superimposed)
ax[1].annotate("Max Coherency: "+tf.strings.as_string(c[2]).numpy().decode('utf-8')+"\nAvg Drop: " +tf.strings.as_string(c[0]).numpy().decode('utf-8') + "\nAvg Increase: " + tf.strings.as_string(c[1]).numpy().decode('utf-8'), xy=(0.5, -0.2), xycoords="axes fraction", ha="center", va="center", fontsize=12)
ax[1].set_title("CAM")
ax[2].imshow(grad_cam_superimposed)
ax[2].annotate("Max Coherency: "+tf.strings.as_string(gc[2]).numpy().decode('utf-8')+"\nAvg Drop: " +tf.strings.as_string(gc[0]).numpy().decode('utf-8') + "\nAvg Increase: " + tf.strings.as_string(gc[1]).numpy().decode('utf-8'), xy=(0.5, -0.2), xycoords="axes fraction", ha="center", va="center", fontsize=12)
ax[2].set_title("Grad-CAM")
ax[3].imshow(smooth_superimposed)
ax[3].annotate("Max Coherency: "+tf.strings.as_string(sgc[2]).numpy().decode('utf-8')+"\nAvg Drop: " +tf.strings.as_string(sgc[0]).numpy().decode('utf-8') + "\nAvg Increase: " + tf.strings.as_string(sgc[1]).numpy().decode('utf-8'), xy=(0.5, -0.2), xycoords="axes fraction", ha="center", va="center", fontsize=12)
ax[3].set_title("Smooth-Grad-CAM")
ax[4].imshow(score_cam_superimposed)
ax[4].annotate("Max Coherency: "+tf.strings.as_string(sc[2]).numpy().decode('utf-8')+"\nAvg Drop: " +tf.strings.as_string(sc[0]).numpy().decode('utf-8') + "\nAvg Increase: " + tf.strings.as_string(sc[1]).numpy().decode('utf-8'), xy=(0.5, -0.2), xycoords="axes fraction", ha="center", va="center", fontsize=12)
ax[4].set_title("Score-CAM")
ax[5].imshow(eigen_cam_superimposed)
ax[5].annotate("Max Coherency: "+tf.strings.as_string(ec[2]).numpy().decode('utf-8')+"\nAvg Drop: " +tf.strings.as_string(ec[0]).numpy().decode('utf-8') + "\nAvg Increase: " + tf.strings.as_string(ec[1]).numpy().decode('utf-8'), xy=(0.5, -0.2), xycoords="axes fraction", ha="center", va="center", fontsize=12)
ax[5].set_title("Eigen-CAM")
ax[6].imshow(svd_grad_cam_superimposed)
ax[6].annotate("Max Coherency: "+tf.strings.as_string(svgc[2]).numpy().decode('utf-8')+"\nAvg Drop: " +tf.strings.as_string(svgc[0]).numpy().decode('utf-8') + "\nAvg Increase: " + tf.strings.as_string(svgc[1]).numpy().decode('utf-8'), xy=(0.5, -0.2), xycoords="axes fraction", ha="center", va="center", fontsize=12)
ax[6].set_title("SVD-Grad-CAM")
for j in range(7):
    ax[j].axis('off')
plt.show()
fig.savefig('Images For Analysis')